In [83]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from sklearn.svm import SVC

import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers.legacy import Adam, SGD
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.callbacks import EarlyStopping

In [84]:
# Load the data
data = np.load('train.npz')

In [85]:
uid = data['uid']
emb1 = data['emb1']
emb2 = data['emb2']
preference = data['preference']

In [86]:
X = np.concatenate((emb1, emb2), axis=1)
print(X.shape)

(18750, 768)


In [87]:
y = preference
print(y.shape)

(18750,)


In [88]:
# train, val split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.01, random_state=42)

In [89]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(18562, 768)
(188, 768)
(18562,)
(188,)


In [90]:
# Data Augmentation (from 15000 -> 45000 through noise injection)
feature_std1 = np.std(X_train)  
print("Standard deviation of features:", feature_std1)
noise_level1 = 0.1 * feature_std1  
noise1 = np.random.normal(0, noise_level1, X_train.shape)
X_train_noisy_pos = X_train + noise1
X_train_noisy_neg = X_train - noise1

X_train= np.concatenate((X_train, X_train_noisy_pos, X_train_noisy_neg), axis=0)

uid = np.concatenate((uid, uid, uid), axis=0)
y_train = np.concatenate((y_train, y_train, y_train), axis=0)

Standard deviation of features: 0.05102888


In [91]:
print(X_train.shape)
print(y_train.shape)

(55686, 768)
(55686,)


In [92]:
def feature_engineering(X_train):
    # Split X_train into emb1 and emb2
    emb1 = X_train[:, :384]  
    emb2 = X_train[:, 384:]  

    # Calculate the difference between emb1 and emb2
    diff = emb1 - emb2

    # Calculate cosine similarities
    cosine_similarities = np.zeros(emb1.shape[0])
    for i in range(emb1.shape[0]):
        norm_emb1 = emb1[i] / np.linalg.norm(emb1[i])
        norm_emb2 = emb2[i] / np.linalg.norm(emb2[i])
        cosine_similarities[i] = np.dot(norm_emb1, norm_emb2)

    cos_sim_reshaped = cosine_similarities.reshape(-1, 1)

    # Calculate mean and variance
    mean_emb1 = np.mean(emb1, axis=1).reshape(-1, 1)
    var_emb1 = np.var(emb1, axis=1).reshape(-1, 1)
    mean_emb2 = np.mean(emb2, axis=1).reshape(-1, 1)
    var_emb2 = np.var(emb2, axis=1).reshape(-1, 1)

    # Combine all features
    combined_features = np.concatenate([diff, mean_emb1, var_emb1, mean_emb2, var_emb2, cos_sim_reshaped], axis=1)
    X_final = np.concatenate([X_train, combined_features], axis=1)

    return X_final


In [93]:
X_train = feature_engineering(X_train)
print(X_train.shape)

(55686, 1157)


In [94]:
X_val = feature_engineering(X_val)

In [95]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(55686, 1157)
(188, 1157)
(55686,)
(188,)


In [98]:
#1. Logistic Regression
# Initialize the model
LR_model = LogisticRegression(C=10, max_iter = 1000)

# Train the model
LR_model.fit(X_train, y_train)

# Predict on validation set
y_pred = LR_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

Validation Accuracy: 0.9414893617021277


In [82]:
# 2. SVM
# Initialize the SVM model
# You can adjust the kernel and other hyperparameters as needed
SVM_model = SVC(C=2)

# Train the model
SVM_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = SVM_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

Validation Accuracy: 0.9308510638297872


Validation Accuracy: 0.8490666666666666


In [53]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

def ANN(dropout_rate = 0.7, neuron_size = 128):
    NN_model = Sequential([
        Dense(16, kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.7),

        Dense(1, activation='sigmoid')
    ])
    NN_model.compile(optimizer=Adam(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])
    NN_model.fit(X_train, y_train, epochs=50, batch_size=300, validation_data=(X_val, y_val), callbacks=[early_stopping])
    return NN_model
NN_model = ANN()
#NN_model.compile(optimizer=SGD(lr=0.001, momentum=0.8), loss='binary_crossentropy', metrics=['accuracy'])

# Evaluate the model on validation set
_, accuracy = NN_model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy}')

2023-12-04 16:50:39.386565: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-12-04 16:50:39.386807: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-12-04 16:50:39.387415: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-12-04 16:50:39.387658: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-04 16:50:39.388010: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2023-12-04 16:50:40.538088: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


150/150 [==============================] - 1s 6ms/step - loss: 0.4551 - accuracy: 0.8004 - val_loss: 0.4636 - val_accuracy: 0.8840
Epoch 2/50
150/150 [==============================] - 1s 5ms/step - loss: 0.3321 - accuracy: 0.8665 - val_loss: 0.3122 - val_accuracy: 0.8901
Epoch 3/50
150/150 [==============================] - 1s 5ms/step - loss: 0.3052 - accuracy: 0.8795 - val_loss: 0.2647 - val_accuracy: 0.8949
Epoch 4/50
150/150 [==============================] - 1s 5ms/step - loss: 0.2951 - accuracy: 0.8824 - val_loss: 0.2579 - val_accuracy: 0.8960
Epoch 5/50
150/150 [==============================] - 1s 5ms/step - loss: 0.2835 - accuracy: 0.8880 - val_loss: 0.2532 - val_accuracy: 0.8963
Epoch 6/50
150/150 [==============================] - 1s 5ms/step - loss: 0.2795 - accuracy: 0.8880 - val_loss: 0.2539 - val_accuracy: 0.9008
Epoch 7/50
150/150 [==============================] - 1s 5ms/step - loss: 0.2766 - accuracy: 0.8909 - val_loss: 0.2563 - val_accuracy: 0.8949
Epoch 8/50
150/15

In [92]:
from itertools import product
from tensorflow.keras.callbacks import EarlyStopping

# Fine tuning neural nets
def ANN(dropout_rate1, neuron_size1, batch_size, X_train, y_train):
    NN_model = Sequential([
        Dense(neuron_size1, kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(dropout_rate1),

        Dense(1, activation='sigmoid')
    ])
    NN_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    NN_model.fit(X_train, y_train, epochs=30, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=[early_stopping])
    return NN_model

# Parameter grid
param_dist = {
    'dropout_rate1': [0.4, 0.5, 0.6, 0.7],
    'neuron_size1': [16, 32, 64, 128, 256],
    'batch_size': [50, 75, 100, 200, 300]
}


best_accuracy = 0 
best_params = {}

# Iterate over all combinations of parameters
for dropout_rate1, neuron_size1, batch_size in product(param_dist['dropout_rate1'], param_dist['neuron_size1'], param_dist['batch_size']):
    
    model = ANN(dropout_rate1, neuron_size1, batch_size, X_train, y_train)
    
    _, accuracy = model.evaluate(X_val, y_val)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {'dropout_rate1': dropout_rate1, 'neuron_size1': neuron_size1, 'batch_size': batch_size}

    print(f"Dropout1: {dropout_rate1}, Neuron Size1: {neuron_size1}, batch size: {batch_size}, Validation Accuracy: {accuracy}")

print(f'Best Validation Accuracy: {best_accuracy} with parameters {best_params}')


Epoch 1/30
900/900 [==============================] - 6s 6ms/step - loss: 0.3141 - accuracy: 0.8690 - val_loss: 0.2605 - val_accuracy: 0.8928
Epoch 2/30
900/900 [==============================] - 5s 5ms/step - loss: 0.2540 - accuracy: 0.8974 - val_loss: 0.2650 - val_accuracy: 0.8885
Epoch 3/30
900/900 [==============================] - 4s 5ms/step - loss: 0.2380 - accuracy: 0.9040 - val_loss: 0.2545 - val_accuracy: 0.8984
Epoch 4/30
900/900 [==============================] - 4s 5ms/step - loss: 0.2250 - accuracy: 0.9086 - val_loss: 0.2639 - val_accuracy: 0.8947
Epoch 5/30
900/900 [==============================] - 4s 5ms/step - loss: 0.2160 - accuracy: 0.9128 - val_loss: 0.2623 - val_accuracy: 0.8963
Epoch 6/30
900/900 [==============================] - 4s 5ms/step - loss: 0.2079 - accuracy: 0.9166 - val_loss: 0.2612 - val_accuracy: 0.8968
Epoch 7/30
900/900 [==============================] - 4s 5ms/step - loss: 0.2035 - accuracy: 0.9187 - val_loss: 0.2638 - val_accuracy: 0.8909
Epoch 

118/118 [==============================] - 1s 4ms/step - loss: 0.2542 - accuracy: 0.8989
Dropout1: 0.4, Neuron Size1: 64, batch size: 100, Validation Accuracy: 0.8989333510398865
Epoch 1/30
225/225 [==============================] - 2s 8ms/step - loss: 0.2923 - accuracy: 0.8748 - val_loss: 0.3580 - val_accuracy: 0.8960
Epoch 2/30
225/225 [==============================] - 2s 10ms/step - loss: 0.2111 - accuracy: 0.9159 - val_loss: 0.2541 - val_accuracy: 0.8957
Epoch 3/30
225/225 [==============================] - 2s 7ms/step - loss: 0.1767 - accuracy: 0.9312 - val_loss: 0.2538 - val_accuracy: 0.8968
Epoch 4/30
225/225 [==============================] - 1s 7ms/step - loss: 0.1535 - accuracy: 0.9414 - val_loss: 0.2648 - val_accuracy: 0.8965
Epoch 5/30
225/225 [==============================] - 1s 6ms/step - loss: 0.1346 - accuracy: 0.9478 - val_loss: 0.2682 - val_accuracy: 0.8941
Epoch 6/30
225/225 [==============================] - 2s 8ms/step - loss: 0.1155 - accuracy: 0.9582 - val_loss

Epoch 8/30
118/118 [==============================] - 1s 4ms/step - loss: 0.2521 - accuracy: 0.8987
Dropout1: 0.4, Neuron Size1: 128, batch size: 300, Validation Accuracy: 0.8986666798591614
Epoch 1/30
900/900 [==============================] - 5s 5ms/step - loss: 0.2547 - accuracy: 0.8930 - val_loss: 0.2596 - val_accuracy: 0.8971
Epoch 2/30
900/900 [==============================] - 4s 5ms/step - loss: 0.1623 - accuracy: 0.9352 - val_loss: 0.2714 - val_accuracy: 0.8923
Epoch 3/30
900/900 [==============================] - 4s 5ms/step - loss: 0.1114 - accuracy: 0.9584 - val_loss: 0.2887 - val_accuracy: 0.8875
Epoch 4/30
900/900 [==============================] - 4s 5ms/step - loss: 0.0834 - accuracy: 0.9695 - val_loss: 0.3129 - val_accuracy: 0.8899
Epoch 5/30
900/900 [==============================] - 4s 5ms/step - loss: 0.0648 - accuracy: 0.9769 - val_loss: 0.3259 - val_accuracy: 0.8944
Epoch 6/30
118/118 [==============================] - 1s 5ms/step - loss: 0.2596 - accuracy: 0.8971

118/118 [==============================] - 1s 5ms/step - loss: 0.2486 - accuracy: 0.8971
Dropout1: 0.5, Neuron Size1: 16, batch size: 75, Validation Accuracy: 0.8970666527748108
Epoch 1/30
450/450 [==============================] - 4s 7ms/step - loss: 0.3481 - accuracy: 0.8518 - val_loss: 0.2706 - val_accuracy: 0.8920
Epoch 2/30
450/450 [==============================] - 2s 5ms/step - loss: 0.2712 - accuracy: 0.8917 - val_loss: 0.2542 - val_accuracy: 0.8944
Epoch 3/30
450/450 [==============================] - 2s 5ms/step - loss: 0.2555 - accuracy: 0.9002 - val_loss: 0.2545 - val_accuracy: 0.8955
Epoch 4/30
450/450 [==============================] - 2s 5ms/step - loss: 0.2473 - accuracy: 0.9031 - val_loss: 0.2518 - val_accuracy: 0.8965
Epoch 5/30
450/450 [==============================] - 2s 5ms/step - loss: 0.2403 - accuracy: 0.9049 - val_loss: 0.2500 - val_accuracy: 0.9008
Epoch 6/30
450/450 [==============================] - 2s 5ms/step - loss: 0.2300 - accuracy: 0.9089 - val_loss: 

Epoch 5/30
150/150 [==============================] - 1s 5ms/step - loss: 0.1642 - accuracy: 0.9338 - val_loss: 0.2526 - val_accuracy: 0.8997
Epoch 6/30
150/150 [==============================] - 1s 5ms/step - loss: 0.1468 - accuracy: 0.9423 - val_loss: 0.2656 - val_accuracy: 0.8944
Epoch 7/30
150/150 [==============================] - 1s 5ms/step - loss: 0.1359 - accuracy: 0.9470 - val_loss: 0.2656 - val_accuracy: 0.8968
Epoch 8/30
118/118 [==============================] - 1s 5ms/step - loss: 0.2498 - accuracy: 0.8981
Dropout1: 0.5, Neuron Size1: 64, batch size: 300, Validation Accuracy: 0.8981333374977112
Epoch 1/30
900/900 [==============================] - 6s 6ms/step - loss: 0.2745 - accuracy: 0.8840 - val_loss: 0.2491 - val_accuracy: 0.8989
Epoch 2/30
900/900 [==============================] - 4s 5ms/step - loss: 0.2015 - accuracy: 0.9167 - val_loss: 0.2511 - val_accuracy: 0.8939
Epoch 3/30
900/900 [==============================] - 4s 5ms/step - loss: 0.1654 - accuracy: 0.9358 

Epoch 5/30
600/600 [==============================] - 3s 5ms/step - loss: 0.0752 - accuracy: 0.9725 - val_loss: 0.3143 - val_accuracy: 0.8891
Epoch 6/30
600/600 [==============================] - 3s 5ms/step - loss: 0.0614 - accuracy: 0.9777 - val_loss: 0.3400 - val_accuracy: 0.8859
Epoch 7/30
118/118 [==============================] - 1s 5ms/step - loss: 0.2578 - accuracy: 0.8973
Dropout1: 0.5, Neuron Size1: 256, batch size: 75, Validation Accuracy: 0.8973333239555359
Epoch 1/30
450/450 [==============================] - 4s 7ms/step - loss: 0.2610 - accuracy: 0.8893 - val_loss: 0.2569 - val_accuracy: 0.8955
Epoch 2/30
450/450 [==============================] - 2s 5ms/step - loss: 0.1740 - accuracy: 0.9306 - val_loss: 0.2689 - val_accuracy: 0.8907
Epoch 3/30
450/450 [==============================] - 2s 5ms/step - loss: 0.1273 - accuracy: 0.9518 - val_loss: 0.2921 - val_accuracy: 0.8883
Epoch 4/30
450/450 [==============================] - 2s 5ms/step - loss: 0.0920 - accuracy: 0.9667 

225/225 [==============================] - 1s 5ms/step - loss: 0.2083 - accuracy: 0.9176 - val_loss: 0.2548 - val_accuracy: 0.8960
Epoch 7/30
225/225 [==============================] - 1s 5ms/step - loss: 0.1996 - accuracy: 0.9209 - val_loss: 0.2603 - val_accuracy: 0.8992
Epoch 8/30
118/118 [==============================] - 1s 5ms/step - loss: 0.2502 - accuracy: 0.9000
Dropout1: 0.6, Neuron Size1: 32, batch size: 200, Validation Accuracy: 0.8999999761581421
Epoch 1/30
150/150 [==============================] - 3s 12ms/step - loss: 0.3711 - accuracy: 0.8366 - val_loss: 0.4307 - val_accuracy: 0.8885
Epoch 2/30
150/150 [==============================] - 1s 6ms/step - loss: 0.2658 - accuracy: 0.8936 - val_loss: 0.2918 - val_accuracy: 0.8968
Epoch 3/30
150/150 [==============================] - 1s 6ms/step - loss: 0.2411 - accuracy: 0.9026 - val_loss: 0.2506 - val_accuracy: 0.8965
Epoch 4/30
150/150 [==============================] - 1s 5ms/step - loss: 0.2291 - accuracy: 0.9087 - val_loss

Epoch 2/30
600/600 [==============================] - 3s 5ms/step - loss: 0.1942 - accuracy: 0.9210 - val_loss: 0.2586 - val_accuracy: 0.8941
Epoch 3/30
600/600 [==============================] - 3s 5ms/step - loss: 0.1558 - accuracy: 0.9386 - val_loss: 0.2673 - val_accuracy: 0.8936
Epoch 4/30
600/600 [==============================] - 3s 5ms/step - loss: 0.1245 - accuracy: 0.9517 - val_loss: 0.2763 - val_accuracy: 0.8928
Epoch 5/30
600/600 [==============================] - 3s 5ms/step - loss: 0.1028 - accuracy: 0.9615 - val_loss: 0.2917 - val_accuracy: 0.8944
Epoch 6/30
118/118 [==============================] - 1s 5ms/step - loss: 0.2462 - accuracy: 0.8957
Dropout1: 0.6, Neuron Size1: 256, batch size: 75, Validation Accuracy: 0.8957333564758301
Epoch 1/30
450/450 [==============================] - 4s 7ms/step - loss: 0.2798 - accuracy: 0.8805 - val_loss: 0.2582 - val_accuracy: 0.8931
Epoch 2/30
450/450 [==============================] - 3s 6ms/step - loss: 0.1929 - accuracy: 0.9214 

450/450 [==============================] - 2s 5ms/step - loss: 0.2320 - accuracy: 0.9067 - val_loss: 0.2524 - val_accuracy: 0.8984
Epoch 7/30
450/450 [==============================] - 2s 5ms/step - loss: 0.2237 - accuracy: 0.9103 - val_loss: 0.2520 - val_accuracy: 0.8965
Epoch 8/30
450/450 [==============================] - 2s 5ms/step - loss: 0.2184 - accuracy: 0.9146 - val_loss: 0.2555 - val_accuracy: 0.8987
Epoch 9/30
450/450 [==============================] - 2s 5ms/step - loss: 0.2175 - accuracy: 0.9137 - val_loss: 0.2551 - val_accuracy: 0.8968
Epoch 10/30
118/118 [==============================] - 1s 5ms/step - loss: 0.2503 - accuracy: 0.8987
Dropout1: 0.7, Neuron Size1: 32, batch size: 100, Validation Accuracy: 0.8986666798591614
Epoch 1/30
225/225 [==============================] - 3s 11ms/step - loss: 0.3774 - accuracy: 0.8326 - val_loss: 0.3760 - val_accuracy: 0.8923
Epoch 2/30
225/225 [==============================] - 1s 6ms/step - loss: 0.2837 - accuracy: 0.8835 - val_los

225/225 [==============================] - 1s 5ms/step - loss: 0.1804 - accuracy: 0.9280 - val_loss: 0.2576 - val_accuracy: 0.8979
Epoch 6/30
225/225 [==============================] - 1s 5ms/step - loss: 0.1673 - accuracy: 0.9335 - val_loss: 0.2567 - val_accuracy: 0.8973
Epoch 7/30
225/225 [==============================] - 1s 5ms/step - loss: 0.1518 - accuracy: 0.9398 - val_loss: 0.2588 - val_accuracy: 0.8987
Epoch 8/30
118/118 [==============================] - 1s 5ms/step - loss: 0.2536 - accuracy: 0.8968
Dropout1: 0.7, Neuron Size1: 128, batch size: 200, Validation Accuracy: 0.8967999815940857
Epoch 1/30
150/150 [==============================] - 3s 12ms/step - loss: 0.3501 - accuracy: 0.8479 - val_loss: 0.4472 - val_accuracy: 0.8936
Epoch 2/30
150/150 [==============================] - 1s 6ms/step - loss: 0.2475 - accuracy: 0.8980 - val_loss: 0.3092 - val_accuracy: 0.8931
Epoch 3/30
150/150 [==============================] - 1s 6ms/step - loss: 0.2179 - accuracy: 0.9103 - val_los

In [106]:
# Load the test data
test_data = np.load('test.npz')
test_uid = test_data['uid']
test_emb1 = test_data['emb1']
test_emb2 = test_data['emb2']

print(test_emb1.shape)

(6250, 384)


In [107]:
# apply feature engineering on the test data
X_test= np.concatenate((test_emb1, test_emb2), axis=1)
print(X_test.shape)
X_test = feature_engineering(X_test)
print(X_test.shape)

(6250, 768)
(6250, 1157)


In [108]:
predictions = LR_model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int).flatten()

In [109]:
import pandas as pd
results_df = pd.DataFrame({'uid': test_uid, 'preference': predicted_classes})

# Export the DataFrame to a CSV file
results_df.to_csv('LR_predictions4.csv', index=False)